First let's define some code to fetch the data. This takes a few seconds so be patient:

In [1]:
from datetime import date, datetime
from urllib.request import urlopen
from json import loads

class CrimeReport:
    def __init__(self, data):
        """
        Creates a CrimeReport from the data as provided by the crimereports.com/v3 API. This is purely for convienent access (`thing.lat` over `thing["lat"]` and some type checking/safety.

        Attributes:
            - type (str): The incident type name.
            - date (datetime): The datetime of the incident.
            - organization (str): The organization name (always Vic PD in this case).
            - lat (float): The Latitude.
            - lng (float): The Longtitude.

        """
        self.type = str(data["incident_type_name"])
        self.date = datetime.strptime(data["incident_date_time"], "%Y-%m-%dT%H:%M:%SZ")
        self.organization = str(data["org_name"])
        self.address = str(data["address"])
        self.lat = float(data["lat"])
        self.lng = float(data["lng"])
    def __repr__(self):
        """
        Defines a pretty printing for a CrimeReport
        """
        return "CrimeReport(type=\""+self.type+"\", address=\""+self.address+"\", lat=\""+str(self.lat)+"\", lng=\""+str(self.lng)+"\", date=\""+str(self.date)+"\", organization=\""+self.organization+"\")"

def get_crime_reports(start=date(2015,1,1), end=date(2016,3,1)):
    """
    Gets a set of crime reports for the VicPD in roughly the Victoria area from start to end dates. This will take a bit due to rate limiting and the volume of data produced.

    Returns a list of CrimeReports.
    """
    # This covers most of Victoria
    rows = range(2831, 2834)
    cols = range(1286, 1291)
    
    start_param = start.strftime("%Y/%m/%d")
    end_param = end.strftime("%Y/%m/%d")

    crimes = []
    incident_types = ",".join([str(x) for x in range(0,200)])
    
    for row in rows:
        for col in cols:
            #print("Getting row "+str(row)+" col "+str(col))
            url = "https://www.crimereports.com/v3/crime_reports/map/search_by_tile.json?start_date="+start_param+"&end_date="+end_param+"&incident_type_ids="+incident_types+"&org_ids=1550&row="+str(row)+"&column="+str(col)+"&zoom=13&include_sex_offenders=false"
            raw_data = urlopen(url).read().decode("utf-8")
            this_data = loads(raw_data)
            if this_data["crimes"]:
                #print(str(len(this_data["crimes"]))+" crimes found on this tile")
                for item in this_data["crimes"]:
                    crimes.append(CrimeReport(item))
    return crimes

# Get all the reports.
crimes = get_crime_reports()

Now lets get the number of crimes by category:

In [3]:
counts = {}
for crime in crimes:
    if crime.type in counts:
        counts[crime.type] += 1
    else:
        counts[crime.type] = 1 

for item,count in sorted([x for x in counts.items()], key=lambda x: x[1]):
    print("{:30}\t{:5}\t{:05.2f}%".format(item, count, float(count) / len(crimes) * 100))

BUSINESS                      	    1	00.01%
POSSESSION-CANNABIS OVER 30G  	    1	00.01%
POSSESSION-MDMA (ECSTASY)     	    2	00.01%
LIQUOR-SEIZURE                	    2	00.01%
THEFT BICYCLE OVER $5000      	    2	00.01%
THEFT FROM MV OVER $5000      	    3	00.02%
ASSAULT-OTHER PEACE OFFICER   	    3	00.02%
THEFT-SHOPLIFTING OVER $5000  	    4	00.02%
215ALCOH IRP WARN-7DAY        	    4	00.02%
THEFT OTHER VEHICLE U/$5000   	    4	00.02%
THEFT-ELEC/GAS/TEL UNDER $5000	    4	00.02%
TAKE AUTO W/O CONSENT U/$5000 	    4	00.02%
EXCESSIVE SPEED-OVER 40KM     	    5	00.03%
POSSESS-OTH SCHED III DRUG/SUB	    5	00.03%
VI-VEHICLE IMPOUNDMENT        	    6	00.04%
ASSAULT POLICE-W/WEAPON OR CBH	    6	00.04%
TRAFFICK-CANNABIS 3000G & UNDR	    9	00.05%
MISCHIEF OVER $5000           	    9	00.05%
TRAFFICKING-METHAMPHETAMINE   	   10	00.06%
ASSAULT-AGGRAVATED            	   14	00.08%
THEFT OF TRUCK OVER $5000     	   19	00.12%
THEFT-OTHER OVER $5000        	   20	00.12%
POSSESS-OTH SCHED I DRUGS/SUBS	 